In [2]:
import sklearn 
import pandas as pd
import xml.etree.ElementTree as ET

As described in the paper, the first step is to select base classifiers. 
The selected base classifiers are trained with default parameter settings with 10-fold cross-validation.
As input data, the training data set and its ground truth labels, per single modality is used.
For the audio MFCC features, we set NaN values to 0, and calculate the average of each MFCC coefficient over all frames.

# Load input data


# Description:

## Available Data
There are variouse csv files and data files available. It's very messy. 
There is one file called "CoE_dataset_offical_release.zip"! 
We extract this file and use this data included their for now! 

## Meta Data
In the original paper there is no information given what is included in the metadata. 
Looking at the paper describing the data set (Right Inflight? A Dataset for Exploring the Automatic Prediction of Movies Suitable for a Watching Situation
) we found out that as metadata they used language, year published, genre, country, runtime and age rating. We assume, since the author of our paper didn't say otherwise, that they used the same metadata. 

## Visual Data: 
The visual data is provied as a csv file for each movie, containing two rows. According to the paper of the dataset they calculated following visual features, Histogram of Oriented Gradients (HOG) gray, Color Moments, local binary patterns (LBP) and Gray Level Run Length Matrix, but don't say how the csv file represents them. Also as mentioned the csv file just has two rows which would not ad up to the mentioned 4 visual features. __We are treating all values as seperate column!__

## Audio Data: 
Audio features is also provided per movie as a csv file. Each audio feature consits of 12 coefficients for multiple frames.

## Textual Data
The textual data is just one file containing the tdf-idf matrix. The first line are the row names for each word. 
While the columns are the associated movie. __There is no indication to which movie each column belongs! Thus we need to assume this!__

__For now we assume the order is the same as in the df_labled_movies dataframe!!!__



In [62]:

df_labled_movies = pd.read_csv("./data/CoE_dataset/Dev_set/dev_set_groundtruth_and_trailers.csv", sep=';')
del df_labled_movies['trailer']
df_labled_movies = df_labled_movies[['movie','filename', 'goodforairplane']]
display(df_labled_movies.head(10))


### Load Meta Data ###

def load_meta_data( filenames ): 
    
    raw_data = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/XML/{file}.xml'
        with open(file_path) as f: 
            tree = ET.parse(f)
            movie = tree.find('movie')
            
            lang = movie.get('language')
            year = movie.get('year')
            genre = movie.get('genre')
            country = movie.get('country')
            runtime = movie.get('runtime')
            age_rating = movie.get('rated')
             
            raw_data.append( (file,lang,year,genre,country,runtime,age_rating) )
    
    return pd.DataFrame(raw_data, columns=['filename','language','year','genre','country','runtime','rated'])


df_meta_data = load_meta_data( df_labled_movies['filename']  )
display(df_meta_data.head(10))
display(df_meta_data.dtypes)

### Load Visual Data ###

def load_visual_data( filenames ):
    data_list = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/vis_descriptors/{file}.csv'
        df_data = pd.read_csv(file_path,index_col=None, header=None)
        data_list.append(df_data)
        
    return pd.concat(data_list, axis = 0, keys = filenames,names=('filename','vis_data'),  sort=False)

df_visual_data = load_visual_data( df_labled_movies['filename']  )
display(df_visual_data.head(10))

### Load Audio Data ###

def load_audio_data( filenames ):
    data_list = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/audio_descriptors/{file}.csv'
        df_data = pd.read_csv(file_path,index_col=None, header=None)
        data_list.append(df_data)
        
    return pd.concat(data_list, axis = 0, keys = filenames,names=('filename','freq_coeff'),  sort=False)

df_audio_data = load_audio_data( df_labled_movies['filename']  )
display(df_audio_data.head(20))


### Load textual Data ###

def load_text_data(filenames):
    

    data_list = []
    file_path = f'./data/CoE_dataset/Dev_set/text_descriptors/tdf_idf_dev.csv'
    #somehow pandas can not really handle that the first line is row names.(at least I didn't find a better way) 
    # thus we do it a little complicated here
    header_index = pd.read_csv(file_path, index_col=0,nrows=1 ).reset_index().columns
    df_data = pd.read_csv(file_path, header=None, index_col=False,skiprows=1)
    df_data.set_index(header_index, inplace=True)
    df_data.columns = filenames
    return df_data.T #row are should be represented by movie names

df_text_data = load_text_data(df_labled_movies['filename'] )
display(df_text_data.head(20))
display(df_text_data.shape)



,movie,filename,goodforairplane
0,Seventh Son,Seventh_Son,1
1,Welcome to Me,Welcome_to_Me,0
2,The Judge,The_Judge,0
3,Transformers Age of Extinction,Transformers__Age_of_Extinction,0
4,The Normal Heart,The_Normal_Heart,1
5,The Phantom Tollbooth,The_Phantom_Tollbooth,1
6,Andaz Apna Apna,Andaz_Apna_Apna,1
7,Hotel Transylvania,Hotel_Transylvania,1
8,The Matrix,The_Matrix,1
9,Into the Wild,Into_the_Wild,1


,filename,language,year,genre,country,runtime,rated
0,Seventh_Son,English,2014,"Action, Adventure, Fantasy","USA, UK, Canada, China",102 min,PG-13
1,Welcome_to_Me,English,2014,"Comedy, Drama",USA,105 min,R
2,The_Judge,English,2014,Drama,USA,141 min,R
3,Transformers__Age_of_Extinction,English,2014,"Action, Adventure, Sci-Fi","USA, China",165 min,PG-13
4,The_Normal_Heart,English,2014,Drama,USA,132 min,TV-MA
5,The_Phantom_Tollbooth,English,1970,"Family, Adventure, Animation",USA,90 min,G
6,Andaz_Apna_Apna,Hindi,1994,"Comedy, Family, Romance",India,160 min,PG
7,Hotel_Transylvania,English,2012,"Animation, Comedy, Family",USA,91 min,PG
8,The_Matrix,English,1999,"Action, Sci-Fi","USA, Australia",136 min,R
9,Into_the_Wild,"English, Danish",2007,"Adventure, Biography, Drama",USA,148 min,R


filename    object
language    object
year        object
genre       object
country     object
runtime     object
rated       object
dtype: object

0        1        2    \
filename                        vis_data                               
Seventh_Son                     0         0.047044  0.11619  0.13633   
                                1         0.056526  0.12516  0.14628   
Welcome_to_Me                   0         0.307170  0.33422  0.33112   
                                1         0.304660  0.33193  0.33124   
The_Judge                       0         0.000000  0.00000  0.00000   
                                1         0.000000  0.00000  0.00000   
Transformers__Age_of_Extinction 0         0.199960  0.26934  0.27986   
                                1         0.189130  0.25738  0.27465   
The_Normal_Heart                0         0.000000  0.00000  0.00000   
                                1         0.000000  0.00000  0.00000   

                                               3         4         5    \
filename                        vis_data                                 
Seventh_Son                     0         0.066194  0.072554  0.172670   
                                1         0.082497  0.079331  0.175380   
Welcome_to_Me                   0         0.331240  0.311140  0.336440   
                                1         0.331380  0.307880  0.332700   
The_Judge                       0         0.000000  0.000000  0.000000   
                                1         0.000000  0.000000  0.000000   
Transformers__Age_of_Extinction 0         0.237250  0.308440  0.332420   
                                1         0.236640  0.303320  0.329890   
The_Normal_Heart                0         0.000000  0.038749  0.083701   
                                1         0.000000  0.201350  0.297900   

                                              6         7         8    \
filename                        vis_data                                
Seventh_Son                     0         0.21519  0.070574  0.071423   
                                1         0.21839  0.093521  0.074837   
Welcome_to_Me                   0         0.33616  0.344790  0.169830   
                                1         0.33357  0.343050  0.173300   
The_Judge                       0         0.00000  0.000000  0.000000   
                                1         0.00000  0.000000  0.000000   
Transformers__Age_of_Extinction 0         0.32998  0.325000  0.307350   
                                1         0.32888  0.322460  0.305430   
The_Normal_Heart                0         0.10544  0.121500  0.038749   
                                1         0.39682  0.553360  0.201350   

                                               9      ...           816  \
filename                        vis_data              ...                 
Seventh_Son                     0         0.149380    ...        731.69   
                                1         0.150250    ...        689.95   
Welcome_to_Me                   0         0.273790    ...        394.34   
                                1         0.280760    ...        397.26   
The_Judge                       0         0.000000    ...     230400.00   
                                1         0.000000    ...     230400.00   
Transformers__Age_of_Extinction 0         0.334310    ...       1112.60   
                                1         0.335510    ...       1120.60   
The_Normal_Heart                0         0.083701    ...      34463.00   
                                1         0.297900    ...      41786.00   

                                                817        818        819  \
filename                        vis_data                                    
Seventh_Son                     0            502.01   1.897000   2.278800   
                                1            474.97   2.267600   2.588700   
Welcome_to_Me                   0            167.91  20.337000  21.276000   
                                1            168.23  20.426000  21.300000   
The_Judge                       0         119950.00  

0      1      2      3      4      5      6      \
filename      freq_coeff                                                    
Seventh_Son   0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              6             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              7             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              8             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              9             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              10            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              11            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              12            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
Welcome_to_Me 0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              6             NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                             7         8         9      ...    23078  23079  \
filename      freq_coeff                                ...                   
Seventh_Son   0          -51.23500 -25.77500 -17.41000  ...      NaN    NaN   
              1            4.76010  10.41400  15.93500  ...      NaN    NaN   
              2           -8.65190  -6.16670  -7.37720  ...      NaN    NaN   
              3           -8.13970  -8.09110 -14.56800  ...      NaN    NaN   
              4           -1.72450   2.19680   1.14500  ...      NaN    NaN   
              5            0.93079   9.88010  15.52800  ...      NaN    NaN   
              6           -2.20740   4.47300   6.56920  ...      NaN    NaN   
              7           -2.63550  -1.67510  -6.13000  ...      NaN    NaN   
              8           -0.33020   1.28230  -0.95568  ...      NaN    NaN   
              9            0.25014   7.69770  11.97200  ...      NaN    NaN   
              10          -1.42530   6.37270  10.30700  ...      NaN    NaN   
              11          -1.03030   0.39305  -1.11160  ...      NaN    NaN   
              12           0.41208  -2.07840  -4.61110  ...      NaN    NaN   
Welcome_to_Me 0                NaN       NaN       NaN  ...      NaN    NaN   
              1                NaN       NaN       NaN  ...      NaN    NaN   
              2                NaN       NaN       NaN  ...      NaN    NaN   
              3                NaN       NaN       NaN  ...      NaN    NaN   
              4                NaN       NaN       NaN  ...      NaN    NaN   
              5                NaN       NaN       NaN  ...      NaN    NaN   
              6                NaN       NaN       NaN  ...      NaN    NaN   

                          23080  23081  23082  23083  23084  23085  23086  \
filename      freq_coeff                                                    
Seventh_Son   0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN 

,24000,baby,baseball,big,doc,escort,frozen,heroes,high,huck,...,years.1,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists
filename,,,,,,,,,,,,,,,,,,,,,
Seventh_Son,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Welcome_to_Me,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Judge,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Transformers__Age_of_Extinction,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Normal_Heart,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.051657,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Phantom_Tollbooth,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Andaz_Apna_Apna,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Hotel_Transylvania,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.041679,0.0,0.0,0.0,0.0
The_Matrix,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.159570,0.159570,0.000000,0.0,0.0,0.0,0.0


(95, 3283)

# Preprocess Data

## Description 

Beside a short description for the audio data there is no more information on how to handle the other data. For example the runtime currently is not handles as a number but as a string(object)
Since sklearn mostly expects numerical inputs, we need to encode the data. 

For different class normally you would use one-hot-encoding, but since it's not specified let's try first the easiest approach which is Labelencoding.


### Audio Data: 
As mentiones in the paper, NaN values of the audio data are set to 0 and the average of each MFCC coefficient is calculated over all frames.





In [32]:

def pre_process_audio_data():
    df_data = df_audio_data.fillna(0.0)
    return df_data.mean(axis=1)
    
    
df_audio_smean = pre_process_audio_data()
display(df_audio_processed.head(20))


filename       freq_coeff
Seventh_Son    0             33.737346
               1             -2.259660
               2              0.822080
               3             -0.298483
               4              0.680520
               5             -0.679905
               6              0.085080
               7             -0.249879
               8             -0.025137
               9             -0.134721
               10            -0.116094
               11            -0.098648
               12             0.066234
Welcome_to_Me  0             39.561047
               1             -4.593651
               2             -0.709224
               3             -1.020713
               4              0.160524
               5              0.001964
               6             -1.487054
dtype: float64

# Define Models

## Description 
These are the models described in the paper. It is not allways clear which exact models they used. (see comments)

In [36]:
from  sklearn.neighbors import KNeighborsClassifier, NearestCentroid #(not sure if this is the nearest mean classifiert) 
from  sklearn.tree import DecisionTreeClassifier
from  sklearn.linear_model import LogisticRegression
from  sklearn.svm import SVC #(not clear which SVC, there is also NuSVC )
from  sklearn.ensemble import BaggingClassifier
from  sklearn.ensemble import AdaBoostClassifier
from  sklearn.ensemble import GradientBoostingClassifier
from  sklearn.ensemble import RandomForestClassifier
from  sklearn.naive_bayes import GaussianNB # there are 3 different naive bayes classifiers, it is not stated which one they used 


model_list = [KNeighborsClassifier(),
                    DecisionTreeClassifier(),
                    LogisticRegression(),
                    SVC(),
                    BaggingClassifier(),
                    AdaBoostClassifier(),
                    GradientBoostingClassifier(),
                    RandomForestClassifier(),
                    GaussianNB() 
                   ]

    


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Define Performance measures:

As mentioned in the paper the performant measueres are the following Precision and Recall and F1-Score. To be more precise the weighted average of Precision and Recall and F1-Score as stated in the dataset paper. 

In [165]:
from sklearn.model_selection import cross_validate

def calculate_metrics(clf,X,y ):
    metric =  cross_validate(clf, X, y, scoring=('precision_weighted','recall_weighted','f1_weighted'), return_train_score=False, cv=10)  
    return pd.Series({'precision':metric['test_precision_weighted'].mean(),'recall':metric['test_recall_weighted'].mean(),'F1':metric['test_f1_weighted'].mean() })

# Select Models

As defined in the paper they use 10-fold CV on the classifiers for training and keep all the classifiers where the metrics are above 0.5 for later stacking.


In [168]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import numpy as np

class MultiColumnLabelEncoder:
    
    def __init__(self, columns = None):
        self.columns = columns # list of column to encode

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        
        output = X.copy()
        
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname, col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        
        return output

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

def getModelName( object ): 

    if hasattr(object, '__module__') and hasattr(object, '__name__'):
        return  object.__name__
    elif hasattr(object, '__module__') and hasattr(object, '__class__'):
        return  object.__class__.__name__
    else:
        raise TypeError("Could not get name of object!")
    
def evaluate_models( X, y ):
    k_fold = KFold(10, random_state=123)
    label_encoder = MultiColumnLabelEncoder(['language','year','genre','country','runtime','rated'])
    metrics = pd.DataFrame()
    
    X_encoded = label_encoder.fit_transform(X)

    for model in model_list:

        m = calculate_metrics(model,X_encoded,y )
        metrics[getModuleFullName(model)] = m

    display(metrics.T)
    
df_train = pd.merge(df_labled_movies,df_meta_data, on='filename')
df_train.drop(['movie', 'filename'],axis=1, inplace=True)
display(df_train.head(2))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']
evaluate_models(df_X, df_y)

,goodforairplane,language,year,genre,country,runtime,rated
0,1,English,2014,"Action, Adventure, Fantasy","USA, UK, Canada, China",102 min,PG-13
1,0,English,2014,"Comedy, Drama",USA,105 min,R


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py

,precision,recall,F1
sklearn.neighbors.classification.KNeighborsClassifier,0.519400,0.566162,0.523350
sklearn.tree.tree.DecisionTreeClassifier,0.500339,0.508687,0.484032
sklearn.linear_model.logistic.LogisticRegression,0.587637,0.585556,0.574014
sklearn.svm.classes.SVC,0.297467,0.536869,0.382622
sklearn.ensemble.bagging.BaggingClassifier,0.572929,0.540000,0.522397
sklearn.ensemble.weight_boosting.AdaBoostClassifier,0.516002,0.501717,0.490721
sklearn.ensemble.gradient_boosting.GradientBoostingClassifier,0.490271,0.489596,0.476228
sklearn.ensemble.forest.RandomForestClassifier,0.454903,0.452222,0.438951
sklearn.naive_bayes.GaussianNB,0.474880,0.506667,0.481515
